## 환경설정

In [70]:
import os

from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("obsidian-rag", set_enable=True)
os.environ["TOKENIZERS_PARALLELISM"] = "true"

LangSmith 추적을 시작합니다.
[프로젝트명]
obsidian-rag


## 문서 로더

In [2]:
from document_loaders.obsidian import MyObsidianLoader

docs_path = "./obsidian-help/ko"
loader = MyObsidianLoader(docs_path, encoding="utf-8", collect_metadata=True)
documents = loader.load()
print(len(documents))

129


## 임베딩 모델

In [3]:
# embedding
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': False}
underlying_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/anpigon/Library/Caches/pypoetry/virtualenvs/obsidian-rag-FsuvQ4pV-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## 임베딩 캐시

https://python.langchain.com/docs/how_to/caching_embeddings/

In [7]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

store = LocalFileStore("./.cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, 
    namespace=underlying_embeddings.model_name,
)

## 문서 분할

시멘틱 청크

In [8]:
# text_splitter
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(cached_embedder)

splitted_docs = text_splitter.split_documents(documents)
print(splitted_docs[0])
print(len(splitted_docs))

page_content='# 옵시디언 도움말

공식 Obsidian 도움말 사이트에 오신 것을 환영합니다. 여기에서 [Obsidian](https://obsidian.md/) 사용 팁 및 가이드를 찾을 수 있습니다. API 문서는 [Obsidian 개발자 문서](https://docs.obsidian.md/)에서 확인하실 수 있습니다. 다음 언어로도 이 사이트를 둘러보실 수 있습니다:

<select class="dropdown select-location">
<option value="">English</option>
<option value="https://publish.obsidian.md/help-ar">العربية</option>
<option value="https://publish.obsidian.md/help-da">Dansk</option>
<option value="https://publish.obsidian.md/help-es">Español</option>
<option value="https://publish.obsidian.md/help-it">Italiano</option>
<option value="https://publish.obsidian.md/help-ja">日本語</option>
<option value="https://publish.obsidian.md/help-km">Phéasa Khmêr</option>
<option value="https://publish.obsidian.md/help-ko">한국어</option>
<option value="https://publish.obsidian.md/help-pt-br">Português</option>
<option value="https://publish.obsidian.md/help-ru">Русский</option>
<option value="https://publish.obsidian.md/help-vi">Tiếng Việt</option>
<option valu

## 벡터스토어

임베딩 인덱싱

In [15]:
from langchain_chroma import Chroma

vector_store_chroma = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./.vectorstore/chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [61]:
vector_store_chroma.reset_collection()
ids = vector_store_chroma.add_documents(splitted_docs)

## Kiwi BM25

한국어 형태소 분석기 키위(Kiwi)

In [65]:
from langchain_teddynote.retrievers import KiwiBM25Retriever

kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(documents=splitted_docs, k=1)

/Users/anpigon/Library/Caches/pypoetry/virtualenvs/obsidian-rag-FsuvQ4pV-py3.11/lib/python3.11/site-packages/langchain_teddynote/retrievers/__init__.py:14: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .kiwi_bm25 import KiwiBM25Retriever


In [66]:
# save kiwi_bm25_retriever
import pickle

with open('./.cached/kiwi_bm25_retriever.pickle', 'wb') as file:
    pickle.dump(kiwi_bm25_retriever, file)

## 앙상블 리트리버(EnsembleRetriever)

In [76]:
from langchain.retrievers import EnsembleRetriever

vectorstore_retriever = vector_store.as_retriever(search_kwargs={"k": 150})

kiwi_bm25_retriever.k = 150

retriever = EnsembleRetriever(
    retrievers=[kiwi_bm25_retriever, vectorstore_retriever],
    weights=[0.6, 0.4],
    search_type="mmr",
)

## 리랭커(Rerank)

In [77]:
# Cohere reranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=20)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

In [78]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="anpigon/qwen2.5-7b-instruct-kowiki:q8_0", temperature=0)

In [79]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [80]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template("""
You are a helpful, respectful and honest assistant for question-answering tasks. 
Always answer as helpfully as possible, while being safe.  
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal information. 
Please ensure that your responses are socially unbiased and positive in nature.
Use the following pieces of retrieved CONTEXT to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer me in Korean. 

<CONTEXT>
{context}      
</CONTEXT>       

Question: {question}
Answer:                                                                                                                                  
""")

compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=20)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## LLM Cache

https://python.langchain.com/docs/how_to/chat_model_caching/

In [126]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache

set_llm_cache(SQLiteCache(database_path="./.cached/llm_cache.db"))
# set_llm_cache(None)

In [81]:
%%time
print(rag_chain.invoke("옵시디언 노트를 싱크하는 방법은?"))

옵시디안 노트를 싱크하는 방법은 다음과 같습니다:

1. **Obsidian Sync 사용**: 이는 디바이스 간에 노트를 동기화하기 위한 안전하고 안심할 수 있는 방법입니다.
2. **다른 클라우드 서비스와 함께 사용**: Dropbox, Google Drive 또는 OneDrive와 같은 다른 클라우드 스토리지 공급자와 함께 Obsidian Sync를 사용하여 노트를 싱크할 수 있습니다.
3. **Obsidian Publish 사용**: 이는 노트를 웹사이트, 위키, 문서 또는 디지털 정원으로 출판하는 방법입니다.

이러한 방법들을 통해 다양한 장치 간에 노트를 최신 상태로 유지할 수 있습니다.


In [164]:
import os

from langchain_core.runnables import ConfigurableField
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)

llama3 = ChatGroq(model="llama3-70b-8192", temperature=0.0)

gpt4o = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

ollama = ChatOllama(model="antegral/llama-varco", temperature=0.0)

openrouter = ChatOpenAI(
    model="liquid/lfm-40b",
    temperature=0.0,
    base_url="https://openrouter.ai/api/v1/",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)

# Initialize LLMs with configuration
llm = gemini.configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="default",
    gemini=gemini,
    gpt4o=gpt4o,
    openrouter=openrouter,
    llama3=llama3,
    ollama=ollama,
).configurable_fields(
    temperature=ConfigurableField(id="temperature"),
)

In [161]:
llm.with_config(configurable={"model": "gemini"}).invoke(
    "당신은 누구입니까?",
    config={"configurable": {"temperature": 1}},
)

AIMessage(content='저는 Google에서 훈련된 대규모 언어 모델입니다. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-7b01ae61-280a-4e83-b364-0a99dbd219b6-0', usage_metadata={'input_tokens': 9, 'output_tokens': 16, 'total_tokens': 25})

In [165]:
rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(rag_chain.invoke("옵시디언 노트를 싱크하는 방법은?"))

옵시디언 노트를 여러 기기에서 동기화하는 가장 쉬운 방법은 옵시디언 싱크(Obsidian Sync)를 사용하는 것입니다. 옵시디언 싱크는 옵시디언 서버를 이용해 노트를 안전하게 동기화하고, 버전 관리 기능도 제공합니다. 

만약 옵시디언 싱크를 사용하고 싶지 않다면, 다음과 같은 방법을 통해서도 노트를 동기화할 수 있습니다.

* **클라우드 저장 공간 활용:** Dropbox, Google Drive, iCloud Drive, OneDrive 등의 클라우드 저장 공간에 옵시디언 보관함 폴더를 저장하고, 여러 기기에서 해당 폴더에 접근하여 노트를 동기화할 수 있습니다.
* **외부 드라이브 사용:** USB 드라이브 또는 외장 하드 드라이브에 옵시디언 보관함 폴더를 저장하고,  필요할 때마다 해당 드라이브를 연결하여 노트에 접근할 수 있습니다.

옵시디언 싱크 외의 방법을 사용할 경우, 동기화 과정에서 충돌이나 데이터 손실이 발생할 수 있으므로 주의해야 합니다. 

자세한 내용은 옵시디언 공식 도움말을 참고하세요:

* [https://help.obsidian.md/ko/Sync/](https://help.obsidian.md/ko/Sync/) 
* [https://help.obsidian.md/ko/Getting_started/Syncing_your_notes](https://help.obsidian.md/ko/Getting_started/Syncing_your_notes) 



## 할루시네이션 체크

- 업스테이지 API 키 발급: https://console.upstage.ai/api-keys
- 소스코드: [UpstageGroundednessCheck](https://api.python.langchain.com/en/latest/_modules/langchain_upstage/tools/groundedness_check.html#UpstageGroundednessCheck)

In [167]:
from langchain_upstage import UpstageGroundednessCheck

upstage_groundedness_check = UpstageGroundednessCheck()

In [168]:
request_input = {
    "context": format_docs(
        compression_retriever.invoke("옵시디언 노트를 무료료 싱크하는 방법은?")
    ),
    "answer": rag_chain.invoke("옵시디언 노트를 무료료 싱크하는 방법은?"),
}

response = upstage_groundedness_check.invoke(request_input)
print(response) # grounded 또는 notGrounded, notSure

PermissionDeniedError: Error code: 403 - {'error_code': 'API_KEY_NOT_ALLOWED', 'message': {'en': 'You have insufficient credit. Please register a payment method. https://console.upstage.ai/billing'}}

## 프롬프트 개선하기

사용자 질문의 의도를 분석하여 프롬프트를 더 풍부하게 만들어보자.

In [169]:
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def prompt_enhancer(query: str, llm: BaseChatModel):
    prompt = PromptTemplate.from_template("""You are a highly specialised psychologist and linguist, and your task is to predict my intentions and refine your prompts for the most effective responses.
Edit the prompt to be more detailed and form complete sentences.
Write as concise and short as possible in Korean.
Whenever you can create a role, situation, or task within a prompt, do so,
The prompts you type should be limited to four sentences or less. Only type your own answers.

Original prompt: {query}                                                                 
Improved prompt:
""")
    chain = (
        {
            "query": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain.invoke(query)

In [171]:
prompt_enhancer("옵시디언 노트를 무료료 싱크하는 방법은?", llm)

'당신은 Obsidian 유저이고, 여러 기기를 사용하며 노트를 동기화하고 싶은 상황입니다. 무료로 여러 기기에서 Obsidian 노트를 동기화할 수 있는 가장 효과적인 방법을 알려주세요. 단, 각 방법의 장단점도 함께 설명해주세요. \n'

In [89]:
prompt_enhancer("옵시디언 노트를 무료료 싱크하는 방법은?", ChatOllama(model="anpigon/qwen2.5-7b-instruct-kowiki:q8_0"))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


'옵시디언 노트를 무료로 동기화할 수 있는 방법은 무엇인가요?'

In [90]:
prompt_enhancer("옵시디언 노트를 무료료 싱크하는 방법은?", ChatOllama(model="eeve-korean-instruct-10.8b:latest"))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


'옵시디언 노트(Obsidian Notes)를 무료로 동기화하는 방법을 알려줄 수 있나요?'

In [93]:
query = "옵시디언에서 노트를 태그로 검색하려면?"
print('qwen2.5-7b-instruct-kowiki:', prompt_enhancer(query, ChatOllama(model="anpigon/qwen2.5-7b-instruct-kowiki:q8_0")))
print("*" * 20)
print('eeve-korean-instruct-10.8b:', prompt_enhancer(query, ChatOllama(model="eeve-korean-instruct-10.8b:latest")))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


qwen2.5-7b-instruct-kowiki: 옵시디언에서 특정 태그로 노트를 검색하려면 어떻게 해야 하나요?
********************


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


eeve-korean-instruct-10.8b: 옵션스티안에서 노트를 태그로 검색하려면, 다음 단계를 따르세요:

1. 옵션스티안에서 노트 목록을 열거나 새로 생성하세요.
2. 원하는 태그를 입력하여 검색창에 입력하세요.
3. '태그 추가' 또는 '태그 편집' 버튼을 클릭하면 노트에 해당 태그가 자동으로 적용됩니다.
4. 태그 이름을 변경하거나 다른 노트를 추가하려면, 각 노트의 오른쪽 상단에 있는 옵션을 클릭한 후 '태그 수정'을 선택하세요.

특정 키워드나 구문을 검색하고 싶다면, '전체 텍스트' 필드를 사용하여 더 정확한 결과를 얻으실 수 있습니다.


- [ ] 리트리버용 쿼리 생성 함수 작성하기